# Импорты

In [42]:
import itertools
import random
import typing as tp

import networkx as nx
import numpy as np
import sympy.matrices as sp

# Константы

In [43]:
EPS = 1e-5

# Проверка матрицы на полную положительность и неотрицательность

Идентичен коду из ноутбука `total_positivity.ipynb`

In [71]:
def calc_order(n: int):
    for i in range(n):
        for j in range(i, n):
            yield i, j

        for j in range(i + 1, n):
            yield j, i


def check_minor(mat: np.ndarray, i: int, j: int):
    if i == j:
        minor = np.round(np.linalg.det(mat[:(i + 1), :(j + 1)]))
    elif j > i:
        minor = np.round(np.linalg.det(mat[:(i + 1), (j - i):(j + 1)]))
    else:
        minor = np.round(np.linalg.det(mat[(i - j):(i + 1), :(j + 1)]))

    if minor < EPS:
        return False

    return True


def total_positivity_test(mat: np.ndarray):
    n = mat.shape[0]

    for i, j in calc_order(n):
        if not check_minor(mat, i, j):
            return False

    return True


def findsubsets(s: set, n: int):
    return list(itertools.combinations(s, n))


def total_non_negativity_test(mat: np.ndarray):
    n = mat.shape[0]

    indices = set([i for i in range(n)])

    for k in range(n):
        subsets = findsubsets(indices, k + 1)

        for i in subsets:
            for j in subsets:
                submatrix = mat[i, :][:, j]

                if np.round(np.linalg.det(submatrix)) < 0:
                    return False

    return True

# Сущность планарной сети

![NETWORK](planar_network.png)

Планарная сеть представляется в виде графа, каждая вершина которого представляет собой пару $(i, j)$, где $i$ - номер уровня по вертикали, $j$ - номер вершины при нумерации слева направо на данном уровне.
Нумерация (порядок) существенных ребер показан на картинке.

Конструктор класса `PlanarNetwork` в качестве аргументов принимает размер планарной сети `n`, а также взвешивание `weighing` в порядке нумерации существенных ребер (не обязательный аргумент).

Методы:

`set_weighing` -- задаёт взвешивание существенных рёбер в порядке обхода

`get_essent_edges` -- возвращает список существенных ребер

`_get_row_width` -- вычисляет длину уровня

`_calc_weight_sequence` -- по переданной матрице и порядку рассмотрения миноров матрицы восстанавливает взвешивание

`_calc_order` -- выдаёт нужный порядок рассмотрения миноров матрицы

`_next_essent_edge` -- по переданному существенному ребру вычисляет следующее по порядку существенное ребро (см. картинку)

`fill_weights` -- по переданной положительной матрице вычисляет взвешивание планарной сети

`calc_mat` -- вычисляет матрицу по взвешиванию сети

In [72]:
class PlanarNetwork:
    def __init__(self, n: int, weighing: tp.List[float] = None):
        self.network = nx.DiGraph()
        self.n = n
        self.weighing = np.ones(shape=n * n) if not weighing else np.array(weighing)
        self.init_edge = ((1, 2), (1, 3)) if n > 1 else ((1, 1), (1, 2))

        for i in range(1, n + 1):
            last_idx = self._get_row_width(i)
            for j in range(1, last_idx):
                self.network.add_edge((i, j), (i, j + 1), weight=1, label='simple')

        w_iter = iter(self.weighing)

        self.network.add_edge(*self.init_edge, weight=next(w_iter, 0), label='essential')
        edge = self._next_essent_edge(self.init_edge)

        while edge != self.init_edge:
            self.network.add_edge(*edge, weight=next(w_iter, 0), label='essential')
            edge = self._next_essent_edge(edge)

    def set_weighing(self, weighing: tp.List[float]):
        w_iter = iter(weighing)

        self.network[self.init_edge[0]][self.init_edge[1]]['weight'] = next(w_iter, 0)
        edge = self._next_essent_edge(self.init_edge)

        while edge != self.init_edge:
            self.network[edge[0]][edge[1]]['weight'] = next(w_iter, 0)
            edge = self._next_essent_edge(edge)

        self.weighing = np.array(weighing)

    def get_essent_edges(self):
        return [edge for edge in self.network.edges.data() if edge[2]['label'] == 'essential']

    def _get_row_width(self, i: int):
        if i > self.n:
            raise ValueError('Index out of bound')

        if self.n == 1:
            return 2

        return 4 + 2 * (i - 1) if i < self.n else 2 + 2 * (i - 2)

    def _calc_weight_sequence(self, mat: np.ndarray, order: tp.Sequence):
        n = mat.shape[0]
        minors = np.zeros(shape=(n, n))

        prefix = 1

        for minor in order:
            i, j = minor

            if i == j == 0:
                minors[i, j] = mat[i, j]

                prefix = mat[i, j]

                yield mat[i, j]
            elif i == j:
                minors[i, j] = sp.Matrix(mat[:(i + 1), :(j + 1)]).det()

                prefix = minors[i, j] / minors[i - 1, j - 1]

                yield prefix
            elif j > i:
                minors[i, j] = sp.Matrix(mat[:(i + 1), (j - i):(j + 1)]).det()

                ans = minors[i, j] / prefix

                if i >= 1:
                    ans /= minors[i - 1, j - 1]

                prefix *= ans

                yield ans
            else:
                minors[i, j] = sp.Matrix(mat[(i - j):(i + 1), :(j + 1)]).det()

                if i == j + 1:
                    prefix = minors[j, j] if j == 0 else minors[j, j] / minors[j - 1, j - 1]

                ans = minors[i, j] / prefix

                if j >= 1:
                    ans /= minors[i - 1, j - 1]

                prefix *= ans

                yield ans

    def _calc_order(self, n: int):
        for i in range(n):
            for j in range(i, n):
                yield i, j

            for j in range(i + 1, n):
                yield j, i

    def _next_essent_edge(self, edge: tp.Tuple[int, int]):
        u, v = edge

        if n == 1 or u == (self.n, self.n - 1) and v == (self.n, self.n):
            return self.init_edge

        if u == (self.n, self.n - 1):
            return (self.n, self.n - 1), (self.n, self.n)

        if u[0] == self.n:
            return (u[1] + 1, u[1] + 2), (u[1] + 1, u[1] + 3)

        if u[0] > v[0]:
            if u[0] == self.n - 1:
                return (u[0] + 1, u[1] - 1), u

            return (u[0] + 1, u[1]), u

        if v[0] == self.n:
            shift = self._get_row_width(v[0]) + 1 - v[1]

            if shift == self.n - 1:
                return (shift + 1, shift), (shift, shift + 1)

            return (shift + 1, shift + 1), (shift, shift + 1)

        if v[0] >= u[0]:
            if v[0] == self.n - 1:
                return v, (v[0] + 1, v[1] - 1)

            return v, (v[0] + 1, v[1] + 2)

    def fill_weights(self, mat: np.ndarray):
        if mat.shape[0] != self.n:
            raise ValueError('Matrix dimension does not match network size')

        cur_edge = self.init_edge

        for i, w in enumerate(
                self._calc_weight_sequence(
                    mat,
                    self._calc_order(mat.shape[0])
                )
        ):
            self.network[cur_edge[0]][cur_edge[1]]['weight'] = w
            cur_edge = self._next_essent_edge(cur_edge)
            self.weighing[i] = w

    def calc_mat(self):
        mat = np.zeros(shape=(self.n, self.n))

        for i in range(1, self.n + 1):
            for j in range(1, self.n + 1):
                src = (i, 1)
                dst = (j, self._get_row_width(j))

                for path in nx.all_simple_paths(
                        self.network,
                        source=src,
                        target=dst,
                ):
                    prod = 1
                    u = path[0]

                    for v in path[1:]:
                        prod *= self.network[u][v]['weight']
                        u = v

                    mat[i - 1, j - 1] += prod

        return mat

# Тесты

In [73]:
count = 100

for n in range(1, 6):
    network = PlanarNetwork(n)

    for _ in range(count):
        pos_weights = np.random.random(n * n) * 4 + 1

        network.set_weighing(pos_weights.tolist())

        mat = network.calc_mat()

        assert total_non_negativity_test(mat)
        assert total_positivity_test(mat)

        network.fill_weights(mat)

        assert np.linalg.norm(pos_weights - network.weighing) < EPS

for n in range(2, 4):
    network = PlanarNetwork(n)

    for _ in range(count):
        nonneg_weights = np.random.random(n * n) * 4 + 1

        nonneg_weights[random.randint(0, len(nonneg_weights) - 1)] = 0

        network.set_weighing(nonneg_weights.tolist())

        mat = network.calc_mat()

        assert total_non_negativity_test(mat)
        assert not total_positivity_test(mat)